In [1]:
from modsim import *

In [8]:
params = Params(x = 0, #m 
                y = 0, #m
                mass = 142.54e-3, #kg
                angle = 90, #degrees
                diameter = 23.495e-2, #m
                rho = 1.2, #kg/m^3
                C_d = 0.3,
                velocity = 40, #m/s
                t_end = 20) #s

In [6]:
state = State(
    y=0, #y position
    x=0, #x position
    vy=0, #y velocity
    vx=35.7632, #x velocity in meters per second
    vr=2308 #rotational velocity in rpm
)

In [18]:
def make_system(params):
    """Make a system object.
    
    params: Params object with angle, velocity, x, y,
               diameter, duration, g, mass, rho, and C_d
               
    returns: System object
    """
    unpack(params)
    
    theta = np.deg2rad(angle)
    
    # compute x and y components of velocity
    vx, vy = pol2cart(theta, velocity)
    
    # make the initial state
    init = State(x=x, y=y, vx=vx, vy=vy, vr=vr)
    
    # compute area from diameter
    area = np.pi * (diameter/2)**2
    
    return System(params, init=init, area=area)

In [19]:
def drag_force(V, system):
    """Computes drag force in the opposite direction of `V`.
    
    V: velocity
    system: System object with rho, C_d, area
    
    returns: Vector drag force
    """
    unpack(system)
    mag = -rho * V.mag**2 * C_d * area / 2
    direction = V.hat()
    f_drag = mag * direction
    return f_drag

In [20]:
def slope_func(state, t, system):
    """Computes derivatives of the state variables.
    
    state: State (x, y, x velocity, y velocity)
    t: time
    system: System object with g, rho, C_d, area, mass
    
    returns: sequence (vx, vy, ax, ay)
    """
    x, y, vx, vy, vr = state
    unpack(system)

    V = Vector(vx, vy)    
    a_drag = drag_force(V, system) / mass
    
    a = a_drag
    
    return vx, vy, a.x, a.y, ar

In [21]:
def event_func(state, t, system):
    x, y, vx, vy, vr = state
    return x - 18.288 #meters

In [22]:
ts = linspace(0, params.t_end, 101)
results, details = run_ode_solver(state, slope_func, events=event_func, max_step=0.2)

ValueError: It looks like `system` does not contain `init`
                 as a system variable.  `init` should be a State
                 object that specifies the initial condition: